# Sistema de recomendacion de Anime
* Construido por razorhedge (github) para Option utilizando el dataset de kaggle: 
* https://www.kaggle.com/CooperUnion/anime-recommendations-database

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
path = r'../input/'

## Exploracion de Datos
* En este caso, se conoce los tamaños de los dataset por informacion previa, por lo que no se necesita utilizar mayor exploracion de tamaño, sin embargo, se incluira por efectos de estandarizacion. 

In [ ]:
anime = 'anime.csv'
ratings = 'rating.csv'

In [ ]:
anime_path = os.path.join(path, anime)
ratings_path = os.path.join(path,ratings)

In [ ]:
df_anime = pd.read_csv(anime_path)
df_ratings = pd.read_csv(ratings_path) 

In [ ]:
df_anime.shape

In [ ]:
df_ratings.shape

### Observamos que el archivo de ratings tiene todas sus entradas en orden, segun el readme, y por que los valores fueron llenados con -1 en caso de no existir valoracion para el usuario.

In [ ]:
df_ratings.info()

### En el caso del dataset de series, encontramos que existen 62 entradas sin genero, 25 sin tipo, y 230 sin rating.

In [ ]:
df_anime.info()

In [ ]:
n_users = df_ratings.user_id.unique().shape[0]
n_users

In [ ]:
n_items = df_ratings.anime_id.unique().shape[0]
n_items

In [ ]:
print(len(set(df_anime.anime_id)))

* A partir de este analisis sabremos que existen 73515 usuarios quienes evaluaron 11200 Animes distintos de la lista de 12294 animes unicos

* Para modelar la recomendacion extraemos los usuarios con mas valoraciones como prueba (usaremos el 42635)

In [ ]:
df_ratings.groupby('user_id').rating.count()

* Buscamos los valores nulos en genero, tipo y ratings

In [ ]:
df_anime[df_anime['genre'].isnull()].sort_values('members', ascending = False).sample(10)

In [ ]:
df_anime[df_anime['type'].isnull()].sort_values('members', ascending = False).sample(10)

In [ ]:
df_anime[df_anime['rating'].isnull()].sort_values('members', ascending = False).sample(10)

* Procedemos a llenar los valores vacios con valores que podamos trabajar, en el caso de genero y episodios, no podemos llenar los valores faltantes por que no conocemos el dato, por lo que lo dejamos en NaN. 
* En caso de tipo, dada la baja cantidad de series, consideraremos estas como TV. 
* En el caso de rating, veremos si el sistema lo incluye a pesar de estar vacio, ya sea llenandolo con una medida o calculandolo.

In [ ]:
df_anime = df_anime.replace('Unknown', np.nan)
df_anime = df_anime.dropna(how = 'all')
df_anime['type'] = df_anime['type'].fillna('TV')
df_anime['episodes'] = df_anime['episodes'].map(lambda x:np.nan if pd.isnull(x) else int(x))
df_ratings = df_ratings.replace(-1, np.nan)

In [ ]:
df_anime[df_anime['anime_id']==841]

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(style = 'white', palette = 'muted')

* Tomamos una vision general del comportamiento de las columnas del dataset de anime

In [ ]:
sns.pairplot(data=df_anime[['type','rating','episodes','members']].dropna(),hue='type')

* Al analizar los datos de ratings, podemos observar que estos tienen una distribucion leptocurtica (datos agrupados en el centro) distribuida entre 3 y 9, con algunos outliers observables. Veremos si esto se da de acuerdo a la mayor cantidad de gente que ha visto una determinada serie, u otra variable, pero sera necesario normalizar esta variable

* Esto significa que si vamos a llenar los valores faltantes sera necesario llenarlos usando la mediana y no el promedio

In [ ]:
%matplotlib inline
plt.hist(df_anime['rating'].fillna(0))

* Llenamos los valores con la mediana para que el analisis sin rating no pese tanto

In [ ]:
df_anime['rating'] = df_anime['rating'].fillna(df_anime.rating.median())

In [ ]:
plt.hist(df_anime['rating'])

* Notamos que no varia la distribucion de los ratings por lo que la funcion sigma no se vera mayormente afectada, sin embargo esto entregara mayor precision al sistema

* Veamos especificamente cuantas valoraciones de cada tipo existen

In [ ]:
pd.DataFrame(df_ratings.groupby('rating').user_id.count()).reset_index()

* Tenemos que el 80% de las evaluaciones se concentran sobre el valor de 4. Reemplazando el -1 por 0 no cambia esta distribucion.
* Veamos si se mantiene al separarlos por tipo

In [ ]:
sns.boxplot(data = df_anime, y = 'rating', x='type')

* En general vemos que la distribucion por tipo es mas extensa de acuerdo a si es serie o OVA, por lo que se priorizara la categorizacion por genero.

 * Y tambien cuantos usuarios han visto cada serie

In [ ]:
plt.hist(df_ratings.groupby(['anime_id'])['anime_id'].count())

* Esto crea un sesgo en el sistema de recomendacion, ya que este constantemente recomendara series que hayan sido mas vistas sobre las que menos, veremos como podemos resolver esta situacion.

* De acuerdo a una busqueda rapida en Google, y el readme de este dataset que es del 2016, las series sin rating no han sido evaluadas ya que o se encuentran aun en emision o no han sido lanzadas, por lo que trataremos que nuestro sistema tambien sea capaz de recomendarlas. 

* Otro dato curioso, tambien observamos que a mayor cantidad de episodios que tenga una serie, tienden a tener un score parecido, o un comportamiento relativamente lineal, por lo que se espera que el sistema asocie las series largas con otras series largas

In [ ]:
sns.scatterplot( x = df_anime['episodes'], y= df_anime['rating'])

## Representaciones Matriciales

* Unimos los datasets para extraer subsets de entrenamiento

In [ ]:
full_df = pd.merge(df_anime, df_ratings, how = 'right', on ='anime_id', suffixes = ['_avg', '_user'])
full_df.rename(columns = {'rating_user':'user_rating', 'rating_avg':'avg_rating'}, inplace = True)

In [ ]:
full_df.sample(10)

* Extraemos un subset para el filtro colaborativo

In [ ]:
df_col = full_df[['user_id', 'name', 'user_rating']]
df_col.head()

In [ ]:
df_genres_list = df_anime['genre'].str.get_dummies(sep = ', ')

In [ ]:
corr = df_genres_list.corr()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(11, 9))

cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

* A partir de este grafico podemos concluir que si queremos reducir los generos podriamos remapear 'Space' y 'Mecha' como Sci-Fi, junto algunas otras categorias, sin embargo por mantener la logica bayesiana no sera del alcance de este modelo

## Para el filtro de Contenido

In [ ]:
df_genres_list.sample(10)

In [ ]:
df_types_list = pd.get_dummies(df_anime[["type"]])
df_types_list.sample(10)

In [ ]:
df_types_list.sample(10)

In [ ]:
df_feat = df_anime[['members','rating','episodes']]

* Creamos el dataset de entrenamiento final para contenido

In [ ]:
df_features = pd.concat([df_feat,df_genres_list, df_types_list], axis = 1).fillna(0)

* Validamos que el dataset haya quedado bien construido

In [ ]:
df_anime[df_anime['anime_id']==5114]

* Creamos funciones de Apoyo

In [ ]:
def get_nombre_from_index(index):
    return df_anime[df_anime.index == index]['name'].values[0]
def get_id_from_nombre(name):
    return df_anime[df_anime.name == name]['anime_id'].values[0]
def get_index_from_id(anime_id):
    return df_anime[df_anime.anime_id == anime_id].index.values[0]

* Consideraremos el valor promedio de las series que el usuario haya evaluado

In [ ]:
#Obtendremos el promedio de las valoraciones que el usuario ha dado a las series para determinar si le gustan, y le recomendaremos series similares a sus favoritas o mejor valoradas.. 
def get_user_top_list(user):
    df_user = df_ratings[df_ratings['user_id']==user]
    df_rated = df_user.dropna(how = 'any')
    avg =  df_rated.rating.mean() 
    df_toplist = df_rated[df_rated['rating']>= avg].sort_values('rating', ascending = False).head(10)
    return list(df_toplist['anime_id'])
def get_user_viewed_list(user):
    return list(df_ratings[df_ratings['user_id']==user]['anime_id'])

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MaxAbsScaler

## Modelado por KNN (Contenido)

* Se usa k = K+1 siendo K el numero de recomendaciones que se desea obtener, ya que la primera siempre es el mismo dato
* Como tenemos variables dummy binarias vs variables con valor muy alto (episodios, miembros) usaremos la biblioteca MaxAbsScaler para convertir dichos valores en una distribucion 0-1. Equivale a normalizar con funcion Z

In [ ]:
mas = MaxAbsScaler()
df_features2 = mas.fit_transform(df_features)

In [ ]:
k = 11

In [ ]:
neighbors_content = NearestNeighbors(n_neighbors = k, algorithm = 'ball_tree')

In [ ]:
neighbors_content.fit(df_features2)

In [ ]:
distances, indices = neighbors_content.kneighbors(df_features2)

In [ ]:
distances.shape

In [ ]:
indices.shape

In [ ]:
from numpy import random

* Extraeremos una serie al azar para evaluar el modelo y ver como se comporta

In [ ]:
series = np,random.randint(0,len(indices))
print(series[1])
name = get_nombre_from_index(series[1])
print(name)

In [ ]:
aid = get_id_from_nombre(name)

In [ ]:
ind = get_index_from_id(aid)

In [ ]:
anime = ind
list(indices[anime,1:11])

In [ ]:
def get_recommendations(aid):
    anime =  get_index_from_id(aid)
    test = list(indices[anime,1:11])
    nb = []
    for i in test:
        a_name = get_nombre_from_index(i)
        nb.append(a_name)
    return nb

### Veremos la comparacion entre la recomendacion y la serie

* Extraemos las series que el usuario ya haya visto

In [ ]:
get_user_top_list(73509)

In [ ]:
get_recommendations(23283)

* Por ultimo recomendamos series hasta que encontremos n series que el usuario no ha visto

In [ ]:
def get_n_recommends(user, n):
    vistas = list(get_user_viewed_list(user))
    liked = list(get_user_top_list(user))
    lista = []
    for i in liked:
        ani = pd.Series(get_recommendations(i))
        recs = np.setdiff1d(ani, vistas) 
        lista.extend(recs)
        if(len(lista) > n):
            lista = lista[n:]
            break
    return lista

* Y probamos para un usuario cualquiera de acuerdo a la lista anterior

In [ ]:
get_n_recommends(10,5)

In [ ]:
get_n_recommends(73509, 10)

* En caso de existir menos series recomendadas ya que el usuario ha visto muy pocas no se rellenara con contenido relevante. Puede considerarse una posible mejora.

## Conclusiones

* El modelo es capaz de relacionar series en base a su contenido, recomendando contenido que comparte los generos y ratings del mismo. Sera necesario medir como se comporta con outliers via LightFM WARP method.

* En vista de la brevedad del tiempo de construccion, se plantean las siguientes mejoras posibles:

    * Realizar clustering the usuarios para mejorar recomendaciones
    * Extraer un ponderado de filtro colaborativo y filtro de contenido para sacar un score final (R. Lineal o Random Forest)
    * Realizar una prediccion estimada de las series que no poseen rating de acuerdo a su contenido.
    * Utilizar sigmoides en tensorflow. 
    * Se pueden utilizar sugerencias aleatorias en vez de una toplist
    * Se puede normalizar el score de las series para sugerir algunas que puedan estar bajo el promedio